In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import eval_extraction
import openai
import extract
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the data

In [7]:
df = pd.read_pickle("../data/export.pkl")[
    [
        "id",
        "paper___url",
        "paper___raw_text",
        "participants___male",
        "participants___female",
        "participants___total",
        "participants___white",
        "participants___black",
        "participants___latino",
        "participants___asian",
    ]
]

# Make predictions

Current outputs:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

Targets:
- 'participants___male', 'participants___female', 'participants___total'
- 'participants___white', 'participants___black', 'participants___latino', 'participants___asian'

In [8]:
idxs = df['paper___raw_text'].notna()
texts = df[idxs]['paper___raw_text'].tolist()
extractions = extract.extract_nums_df(texts)
for k in extractions.keys():
    df.loc[idxs, k] = extractions[k]

attempting to add dict_keys(['num_male', 'num_female', 'num_total', 'evidence_span_gender'])


0it [00:00, ?it/s]

538it [00:00, 1200.32it/s]


attempting to add dict_keys(['num_white', 'num_black', 'num_latino', 'num_asian', 'evidence_span_race'])


538it [00:00, 1319.00it/s]

Unterminated string starting at: line 6 column 23 (char 105)


### Evaluate
Evaluates whether each extracted number is within 1 of the human-labeled value

In [9]:
# Apply some postprocessing (convert percentages)
gender_count_processed = df.apply(eval_extraction.process_gender_counts, axis=1)
df["num_male"] = gender_count_processed.apply(lambda x: x[0]).astype(str)
df["num_female"] = gender_count_processed.apply(lambda x: x[1]).astype(str)

print("Total n (with paper text)", idxs.sum())
eval_extraction.compute_metrics(
    df,
    preds_col_to_gt_col_dict={
        "num_male": "participants___male",
        "num_female": "participants___female",
        "num_total": "participants___total",
    },
)

Total n (with paper text) 538


,target,recall,n_labeled,n_predicted_num
0,num_male,191,221,253
1,num_female,191,221,256
2,num_total,219,259,386
